<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/Google_gemma_LORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference
- ref. https://gist.github.com/alfredplpl/e20cad036c151f38645a1abc87f56a2f
- ref. https://huggingface.co/datasets/kunishou/OpenMathInstruct-1-1.8m-ja
- ref. https://note.com/npaka/n/nc55e44e407ff
- ref. https://huggingface.co/blog/gemma-peft
Licence: MIT

In [ ]:
!pip install -U transformers
!pip install -U peft
!pip install accelerate
!pip install datasets
!pip install trl
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:


from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
import os

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=HF_TOKEN)

# model = AutoModelForCausalLM.from_pretrained(model_id, token=HF_TOKEN)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

# データセットの読み込み

In [ ]:
dataset = load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja", split="train")
dataset[0:2]

{'index': [0, 1],
 'dataset': ['gsm8k', 'gsm8k'],
 'question': ['Martha has 18 crayons. She lost half of them, so she bought a new set of 20 crayons. How many crayons in total does Martha have after the purchase?',
  'Noa scored 30 points to win a bowl, and Phillip scores twice that number. What is the total number of points Tom and Phillip scored to win the bowl?'],
 'generated_solution': ["Let's solve this problem using Python code.\n<llm-code>\namount_of_lost_crayons = 18 / 2\namount_of_new_crayons = 20\ntotal_amount = amount_of_lost_crayons + amount_of_new_crayons\ntotal_amount\n</llm-code>\n<llm-code-output>\n29.0\n</llm-code-output>\nThus Martha has \\\\boxed{29} crayons in total.",
  "Let's solve this problem using Python code.\n<llm-code>\nscore_first_player = 30\nscore_second_player = 2 * score_first_player\nscore_first_player + score_second_player\n</llm-code>\n<llm-code-output>\n90\n</llm-code-output>\nThus Tom and Phillip scored \\\\boxed{90} points."],
 'question_ja': ['マー

In [ ]:
dataset.column_names
remove_cols = ['dataset',
 'question',
 'generated_solution',
 'question_ja',
 'generated_solution_ja']

In [ ]:
# プロンプトの生成
def generate_prompt(example):
    return """<bos><start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<eos>""".format(example["question_ja"], example["generated_solution_ja"])

# textカラムの追加
def add_text(example):
    example["text"] = generate_prompt(example)
    return example
dataset = dataset.map(add_text)

In [ ]:
dataset = dataset.remove_columns(remove_cols)
dataset[0:2]

{'index': [0, 1],
 'text': ['<bos><start_of_turn>user\nマーサは18本のクレヨンを持っている。その半分をなくしてしまったので、新しいクレヨンを20本買いました。購入後、マーサは全部で何本のクレヨンを持っていますか。<end_of_turn>\n<start_of_turn>model\nPythonコードを使用してこの問題を解決しましょう。<llm-code>\namount_of_lost_crayons = 18 / 2\namount_of_new_crayons = 20\ntotal_amount = amount_of_lost_crayons + amount_of_new_crayons\ntotal_amount\n</llm-code><llm-code-output>\n29.0\n</llm-code-output>\nしたがって、マーサは合計で\\\\ boxed {29}本のクレヨンを持っています。<eos>',
  '<bos><start_of_turn>user\nボウルに勝つためにノアは30点、フィリップはその2倍を得点した。ボウルに勝つためにトムとフィリップが得点した合計点数は？<end_of_turn>\n<start_of_turn>model\nPythonコードを使用してこの問題を解決しましょう。<llm-code>\nscore_first_player = 30\nscore_second_player = 2 * score_first_player\nscore_first_player + score_second_player\n</llm-code><llm-code-output>\n90\n</llm-code-output>\nしたがって、トムとフィリップは\\\\ boxed {90}ポイントを獲得しました。<eos>']}

In [ ]:
# 実験なので・・・。
# dataset=dataset.select(range(1000))

In [ ]:
# データセットの分割
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [ ]:
# !rm -Rf /content/outputs

In [ ]:
#
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=50,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    dataset_text_field="text"
)
trainer.train()
trainer.save_model("./model/")

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1642507 [00:00<?, ? examples/s]

Map:   0%|          | 0/182501 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
50,2.445500
100,1.386600
150,1.334700
200,1.243100
250,1.206100
300,1.174800
350,1.187600
400,1.141700
450,1.135100
500,1.140400


In [ ]:
# プロンプトの準備
prompt="""<start_of_turn>user
猫と犬、どっちが好き？<end_of_turn>
<start_of_turn>model
"""
# 推論の実行
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **input_ids,
    max_new_tokens=128,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.1,
)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
猫と犬、どっちが好き？<end_of_turn>
<start_of_turn>model
猫と犬のどちらが好きですか？
これは、猫と犬のどちらが好きなことを意味します。
猫が好きな場合、猫が好きな場合、犬が好きな場合、犬が好きな場合、猫が好きな場合、猫が好きな場合、犬が好きな場合、犬が好きな場合、猫が好きな場合、猫が好きな場合、猫が好きな場合、犬が好きな場合、犬が好きな場合、猫が好きな場合、猫が好きな場合、猫が好きな場合、犬が好きな場合、犬が好きな場合、猫が好きな場合、猫が好きな場合、猫が好きな場合、


In [ ]:
# プロンプトの準備
prompt="""<start_of_turn>user
富士山の高さを求めて、その高さの８０％は何メートル？<end_of_turn>
<start_of_turn>model
"""
# 推論の実行
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **input_ids,
    max_new_tokens=128,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.1,
)
print(tokenizer.decode(outputs[0]))

In [ ]:
# モデルを保存します。
# zip 圧縮する
!zip -r /content/download.zip /content/model

# ダウンロードする
from google.colab import files
files.download("/content/download.zip")

In [ ]:
del model

In [ ]:
model_path = "/content/model"
tokenizer = AutoTokenizer.from_pretrained(model_path, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map={"":0}, token=HF_TOKEN)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# プロンプトの準備
prompt="""<start_of_turn>user
日本で一番高い富士山高さは3,776mメールです。80%まで登ると標高は何m？<end_of_turn>
<start_of_turn>model
"""
# 推論の実行
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **input_ids,
    max_new_tokens=256,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.1,
)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
日本で一番高い富士山高さは3,776mメールです。80%まで登ると標高は何m？<end_of_turn>
<start_of_turn>model
Pythonコードを使用してこの問題を解決しましょう。<llm-code>
# height of the mountain in meters
height_in_meters = 3776
# percentage of the way up
percentage_of_way_up = 80
# height of the mountain in meters when it is 80% of the way up
height_when_it_is_80_percent_of_the_way_up = height_in_meters * percentage_of_way_up / 100
height_when_it_is_80_percent_of_the_way_up
</llm-code><llm-code-output>
2005.0
</llm-code-output>
したがって、富士山は\\Boxed{2005}メートルに80％の高さに到達します。<eos>


In [ ]:
# プロンプトの準備
prompt="""<start_of_turn>user
日本で一番高い富士山高さは３７７６メールです。８０％まで登ると標高は何メートルですか。ステップ・バイ・ステップで考えてください。<end_of_turn>
<start_of_turn>model
"""
# 推論の実行
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **input_ids,
    max_new_tokens=256,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.1,
)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
日本で一番高い富士山高さは３７７６メールです。８０％まで登ると標高は何メートルですか。ステップ・バイ・ステップで考えてください。<end_of_turn>
<start_of_turn>model
Pythonコードを使用してこの問題を解決しましょう。<llm-code>
height_of_mount_fuji = 3776
percent_of_height_to_climb = 80
height_climbed = height_of_mount_fuji * percent_of_height_to_climb / 100
height_climbed
</llm-code><llm-code-output>
2950.0
</llm-code-output>
したがって、富士山は\\ Boxed {2950}メートルに登ります。<eos>


In [ ]:
# プロンプトの準備
prompt="""<start_of_turn>user
500円のりんごを5個と80円のみかんを10個買いました。いくらになりますか？<end_of_turn>
<start_of_turn>model
"""
# 推論の実行
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **input_ids,
    max_new_tokens=256,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.1,
)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
500円のりんごを5個と80円のみかんを10個買いました。いくらになりますか？<end_of_turn>
<start_of_turn>model
Pythonコードを使用してこの問題を解決しましょう。<llm-code>
cost_per_apple = 500 / 5
cost_per_orange = 80
total_cost = cost_per_apple * 5 + cost_per_orange * 10
total_cost
</llm-code><llm-code-output>
2000.0
</llm-code-output>
したがって、合計コストは\\ boxed {2000}ドルです。<eos>
